## Valorant Decision Tree Model

In [2]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sqlalchemy import create_engine, Column, Integer, String, Boolean, Float, ForeignKey
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json

# Base = declarative_base()

## Loading and Preprocessing Agent Data

In [7]:
import sqlite3

# Create database connection
engine = create_engine('sqlite:///SQlite/Valorant.sqlite')

In [9]:
# Querying and creating Dataframe
query_1 = "SELECT characterId FROM player_stats"

agent_df = pd.read_sql(query_1, engine)
agent_df.head()

,characterId
0,320b2a48-4d9b-a075-30f1-1f93a9b638fa
1,569fdd95-4d10-43ab-ca70-79becc718b46
2,eb93336a-449b-9c1b-0a54-a891f7921d69
3,22697a3d-45bf-8dd7-4fec-84a9e28c69d7
4,320b2a48-4d9b-a075-30f1-1f93a9b638fa
...,...
1995,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc
1996,569fdd95-4d10-43ab-ca70-79becc718b46
1997,1e58de9c-4950-5125-93e9-a0aee9f98746
1998,f94c3b30-42be-e959-889c-5aa313dba261


# Building the Linear Regression Model

In [ ]:
# Create a model with scikit-learn
model = LinearRegression()

In [ ]:
# Fit the data into the model
model.fit(X, y)

In [ ]:
# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")

In [ ]:
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

In [ ]:
# Make predictions using the X set
predicted_y_values = model.predict(X)